In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
import matplotlib.pyplot as pl
import matplotlib.colors as mc
import matplotlib.collections as mcoll
from scipy.interpolate import NearestNDInterpolator
from numpy import linspace, array, logspace, sin, cos, pi, arange, sqrt, arctan2, arccos
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from coords import *
from adjustText import adjust_text
import matplotlib.patheffects as PathEffects
from astropy.coordinates import Angle
import astropy.units as u

import lmfit
from lmfit import minimize, Parameters,create_params



from collections import Counter
from scipy.spatial.distance import cdist
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import NearestNeighbors


In [2]:
plt.style.use('classic')
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.weight"] = "bold"
plt.rcParams["font.size"] = "16"
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"

def rotate(x,y,theta):
    xprime = x*cos(theta) - y*sin(theta)
    yprime = x*sin(theta) + y*cos(theta)
    return xprime, yprime

l_offset = np.radians(-0.0558)
b_offset = np.radians(-0.0462)
x_offset = Rsun*l_offset
y_offset = Rsun*b_offset

#############################
# create class that stores a ring
#############################

class Ring:

    def __init__(self,t,a,b,z,v0,theta,xyzsun,vxyzsun,alpha=0.4):
        self.t     = t
        self.a     = a
        self.b     = b
        self.z0    = z
        self.v0    = v0
        self.theta = theta
        self.x     = a*cos(t)
        self.y     = -b*sin(t)
        self.z     = self.z0*sin(-2*t + alpha)
        self.R     = sqrt(self.x**2+self.y**2)
        self.phi   = -arctan2(self.y,self.x)
        self.ephix = -sin(self.phi) # unit vector parallel to circle
        self.ephiy = -cos(self.phi) # unit vector parallel to circle
        norm       = sqrt((a*sin(t))**2+(b*cos(t))**2)
        self.ex    = -a*sin(t)/norm # unit vector parellel to ellipse
        self.ey    = -b*cos(t)/norm  # unit vector parallel to ellipse
        self.cosalpha = self.ex*self.ephix + self.ey*self.ephiy
        self.vphi  = self.R[0]*self.v0/self.R # assume conservation of angular momentum
        self.v     = self.vphi/self.cosalpha # total speed along the orbit
        self.vx    = +self.v*self.ex
        self.vy    = +self.v*self.ey
        self.vz    = np.zeros(t.size)
        self.x,self.y   = rotate(self.x,self.y,theta)
        self.vx,self.vy = rotate(self.vx,self.vy,theta)
        self.X,self.Y,self.Z,self.vX,self.Vy,self.vZ = xyz2XYZ(self.x,self.y,self.z,self.vx,self.vy,self.vz,xyzsun[0],xyzsun[1],xyzsun[2],vxyzsun[0],vxyzsun[1],vxyzsun[2])
        # Convert Sgr A* offset from degrees to radians
        l_offset = np.radians(0.05)
        b_offset = np.radians(-0.0462)
        x_offset = Rsun*l_offset
        y_offset = 0
        z_offset = Rsun*b_offset
        # Calculate galactic coordinates accounting for Sgr A* offset
        self.l,self.b,self.r,self.vl,self.vb,self.vr = xyz2lbr(
            self.x,self.y,self.z,self.vx,self.vy,self.vz,
            xyzsun[0],xyzsun[1],xyzsun[2],vxyzsun[0],vxyzsun[1],vxyzsun[2])
        self.l += l_offset
        self.b += b_offset
        self.x += x_offset
        self.y += y_offset
        self.z += z_offset
        self.mu_l, self.mu_b = vlb_2_mulb(self.r, self.vl*100, self.vb*100)
        self.mu_l, self.mu_b = vlb_2_mulb(self.r,self.vl*100,self.vb*100)

        
#############################
# define sun position & velocity
#############################

xsun  = 0.0
ysun  = -8.1
zsun  = 0.0
vxsun = -2.2
vysun = 0.0
vzsun = 0.0
xyzsun  = [xsun, ysun, zsun ]
vxyzsun = [vxsun,vysun,vzsun]
phisun  = arctan2(ysun,xsun)



In [3]:
def lb_lv_plots(Rings, a, b):
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(9, 6))
    fig.set_facecolor('white')
    ax[0].set_facecolor('white')

    ### Scatter cloud points in lbv ###
    color_list = ['blue', 'cyan', 'gray' ,'mistyrose', 'red']
    cmap = (mpl.colors.ListedColormap(color_list))




    ### Plot Rings LB ###
    ax[0].plot(np.degrees(Rings.l[back_ind[0:-1]]),np.degrees(Rings.b[back_ind[0:-1]]),c='red')
    ax[0].plot(np.degrees(Rings.l[fore_ind[0:]]),np.degrees(Rings.b[fore_ind[0:]]),c='blue', linewidth=2, zorder=2)

    for i in range(0,len(cat_agg)):

        if cat_agg['NF_decision'][i] == 'N':
            ax[0].scatter(cat_agg['l'][i],cat_agg['b'][i], marker='o', c=color_list[0], s=120 )
        if cat_agg['NF_decision'][i] == 'LN':
            ax[0].scatter(cat_agg['l'][i],cat_agg['b'][i], marker='o', c=color_list[1], s=120)
        if cat_agg['NF_decision'][i] == 'U':
            ax[0].scatter(cat_agg['l'][i],cat_agg['b'][i], marker='o', c=color_list[2], s=120)
        if cat_agg['NF_decision'][i] == 'LF':
            ax[0].scatter(cat_agg['l'][i],cat_agg['b'][i], marker='o', c=color_list[3], s=120)
        if cat_agg['NF_decision'][i] == 'F':
            ax[0].scatter(cat_agg['l'][i],cat_agg['b'][i], marker='o', c=color_list[4], s=120)
        if cat_agg['NF_decision'][i] == 'nan':
            ax[0].scatter(cat_agg['l'][i],cat_agg['b'][i], marker='s', s=150, edgecolor='gray', zorder=3, c='k')


    ax[0].set_xlabel('longitude [deg]', labelpad=20)
    ax[0].set_ylabel('latitude [deg]', labelpad=10)
    ax[0].set_xlim(1.8,-1.)
    ax[0].set_ylim(-0.3,0.1)


    ### Plot Rings LV ###
    ax[1].plot(np.degrees(Rings.l[back_ind[0:-1]]),Rings.vr[back_ind[0:-1]],c='red')
    ax[1].plot(np.degrees(Rings.l[fore_ind[0:]]),Rings.vr[fore_ind[0:]],c='blue', linewidth=2, zorder=2)

    for i in range(0,len(cat_tab)):

        if cat_tab['NF_decision'][i] == 'N':
            ax[1].scatter(cat_tab['l'][i],cat_tab['v'][i], marker='o', c=color_list[0], s=120 )
        if cat_tab['NF_decision'][i] == 'LN':
            ax[1].scatter(cat_tab['l'][i],cat_tab['v'][i], marker='o', c=color_list[1], s=120)
        if cat_tab['NF_decision'][i] == 'U':
            ax[1].scatter(cat_tab['l'][i],cat_tab['v'][i], marker='o', c=color_list[2], s=120)
        if cat_tab['NF_decision'][i] == 'LF':
            ax[1].scatter(cat_tab['l'][i],cat_tab['v'][i], marker='o', c=color_list[3], s=120)
        if cat_tab['NF_decision'][i] == 'F':
            ax[1].scatter(cat_tab['l'][i],cat_tab['v'][i], marker='o', c=color_list[4], s=120)
        if cat_tab['NF_decision'][i] == 'nan':
            ax[1].scatter(cat_tab['l'][i],cat_tab['v'][i], marker='s', s=150, edgecolor='gray', zorder=3, c='k')






    ax[1].set_xlabel('longitude [deg]', labelpad=20)
    ax[1].set_ylabel('velocity [km/s]', labelpad=10)
    ax[1].set_xlim(1.8,-1.)
    ax[1].set_ylim(-150,150)
    
    
    ax[0].text(.95, .95, 'a = {:.3f}, b = {:.3f}'.format(a,b), fontsize=13, ha='right', va='top', 
            transform=ax[0].transAxes)
    
    
    plt.show()

    return 

In [4]:
def distance_euclidean(x, y, z, x0, y0, z0):
    d_x = x - x0
    d_y = y - y0
    d_z = z - z0
    dis = np.sqrt( d_x**2 + d_y**2 +d_z**2)
    return dis

#Try this other distance metric later...
def calculate_mahalanobis_distances(data1, data2):
    cov = np.cov(data1.T)
    inv_cov = np.linalg.inv(cov)
    distances = cdist(data1, data2, metric='mahalanobis', VI=inv_cov)
    return distances, inv_cov


def min_distance(ringl, ringb, ringv, P):
    """
    Compute minimum/a distance/s between
    a point P[x0,y0,z0] and a curve (x,y,z).
    
    Returns min indexes and distances array.
    """
    # compute distance
    d = distance_euclidean(np.degrees(ringl), np.degrees(ringb), ringv, P[0], P[1], P[2])

    # find the minima
    glob_min_idxs = np.argwhere(d==np.min(d)).ravel()
    return glob_min_idxs, d



#Gaussian function for the PPDF distributions (used in KNN search)
def gaussian(x, A, mu, sigma):
    return A * np.exp(- (x - mu)**2 / (2.* (sigma**2 )))

In [6]:
def predict_near_far(model_data, catalogue_data, n_neighbors, inv_cov):
    nn = NearestNeighbors(n_neighbors=n_neighbors, metric='mahalanobis', metric_params={'VI': inv_cov})
    nn.fit(model_data[['l', 'b', 'v']].values)
    distances, indices = nn.kneighbors(catalogue_data)

    predicted_nf = []
    weights = []
    for idx, dist in zip(indices, distances):
        neighbors_nf = model_data['near_far'].iloc[idx].values
        predicted_nf.append(Counter(neighbors_nf).most_common(1)[0][0])
        weight = 1 / np.median(dist)
        weights.append(weight)

    return np.array(predicted_nf), np.array(weights)


def analyse_model_4d(model, catalogue, model_name, n_neighbors):
    model_data = model[['l', 'b', 'v']].values
    catalogue_data = catalogue[['l', 'b', 'v']].values

    distances, inv_cov = calculate_mahalanobis_distances(model_data, catalogue_data)
    normalised_distances = distances / np.max(distances)
    closest_indices = np.argmin(normalised_distances, axis=0)
    overall_distance_3d = np.mean(np.min(normalised_distances, axis=0))

    predicted_nf, weights = predict_near_far(model, catalogue_data, n_neighbors, inv_cov)

    actual_nf = catalogue['near_far'].values
    nf_accuracy = np.sum((predicted_nf == actual_nf) * weights) / np.sum(weights)

    combined_score = np.mean([(1 - overall_distance_3d), nf_accuracy])

    errors = catalogue[['l', 'b', 'v']] - model[['l', 'b', 'v']].iloc[closest_indices].values

    return {
        'name': model_name,
        'overall_distance_3d': overall_distance_3d,
        'nf_accuracy': nf_accuracy,
        'combined_score': combined_score,
        'errors': errors,
        'data': model,
    }


def load_and_preprocess_models():
    model_files = [
    ('molinari_resampled_300.txt', '\t', "Molinari"),
    ('sofue_resampled_300.txt', '\t', "Sofue"),
    ('kdl_resampled_300.txt', '\t', "KDL"),
    ('ellipse_resampled_300.txt', '\t', "Ellipse")
]

    return [
        (*preprocess_data(load_data(file, sep)), name)
        for file, sep, name in model_files
    ]


def main():
    catalogue, catalogue_original = preprocess_data(load_data('lipman-catalogue.txt', sep=','))
    models = load_and_preprocess_models()

    n_neighbors = int(round(np.sqrt(len(models[0][0]))))

    results = [
        analyse_model_4d(model, catalogue, name, n_neighbors)
        for model, original_data, name in models
    ]

    for result, (_, original_data, _) in zip(results, models):
        result['original_data'] = original_data

    results.sort(key=lambda x: x['combined_score'], reverse=True)

    print("Model ranking for 4D (l,b,v,n/f) space:")
    for i, result in enumerate(results, 1):
        print(f"{i}. {result['name']} (score: {result['combined_score']:.2f})")



In [5]:
###Read in data to scatter plot in lbv 
cat_tab = Table.read("/Users/danilipman/Documents/Research/UConn/CMZ_SYNTH/synth_table.tex")


cat_index = cat_tab['leaf_id']
cloud_name = cat_tab['cloud_name']
l = cat_tab['l']
b = cat_tab['b']
v = cat_tab['v']
sigma = cat_tab['sigma']
rad = cat_tab['rad']
NF_decision = cat_tab['NF_decision']
lb_pixel_mask = cat_tab['lb_pixel_mask']



## aggregate table that pulls out the largest mask
## to represent the l,b N/F/U position

cat_group = cat_tab.group_by('lb_pixel_mask')
cat_group_mask = cat_group.groups.keys['lb_pixel_mask'] == 1

cat_agg = cat_group.groups[cat_group_mask]


